In [4]:
#pragma cling add_include_path("/home/ilobach/anaconda3/include/")

In [5]:
#include <xtensor/xnpy.hpp>
#include <complex>
#include <cstdlib>
#include <iostream>
#include <map>
#include <string>
#include <cmath>
#include <ctime>

In [3]:
auto data = xt::load_npy<std::complex<double>>("/mnt/c/Users/lobac_000/OneDrive - Fermi National Accelerator Laboratory/FUR/SRW_SLAC_undulator_spectrum/Ex_3D.npy");
    std::cout << data.dimension() << std::endl;

3


In [4]:
data.shape()[2]

1024

In [5]:
int nx, ny, nz;
nx = data.shape()[2];
ny = data.shape()[1];
nz = data.shape()[0];

In [6]:
const std::complex<double> * Ex3d = data.data();

In [7]:
Ex3d[100,300,10]

(-6.92012,-0.677458)

In [8]:
Ex3d[1*(nx*ny)+1*(nx)+1]

(0.782565,-5.8691)

In [9]:
std::complex<double> ex3d(int z, int y, int x) {
    return Ex3d[z*(nx*ny)+y*(nx)+x];
}

In [10]:
ex3d(1,1,1)

(0.782565,-5.8691)

# Inputs:

In [11]:
double Sx = 300.96585801243214;
double Sy = 240.67384278137428;
double dx = 1048234.8735736432;
double dy = -487663.40045622655;
double sxp = 0.00043265807507931146;
double syp = 0.00043148229886993364;

In [12]:
double dax = 4.9609375e-05;
double day = 4.9614674702039514e-05;

In [13]:
double lmin = 0.85;
double dl = 0.0015000000000000002;

In [14]:
double axmin = -dax*(nx-1)/2;
double aymin = -day*(ny-1)/2;

# Inferred values

In [15]:
double V = dl*(nz-1)*pow(dax*(nx-1)*day*(ny-1),3);

In [16]:
int ixp, iyp, m1, n1, m2, n2, il;

In [17]:
int m1Mixp, n1Miyp, m2Mixp, n2Miyp;

In [18]:
bool isinRange(int val, int r) {
    if ((val < 0) || (val > r-1)) {
        return false;
    }
    else {
        return true;
    }
}

In [19]:
bool isin(int a, int b, int c, int d, int l) {
    if (not isinRange(a,nx)){
        return false;
    }
    if (not isinRange(b,ny)) {
        return false;
    }
    if (not isinRange(c,nx)) {
        return false;
    }
    if (not isinRange(d,ny)) {
        return false;
    }
    if (not isinRange(l,nz)) {
        return false;
    }
    return true;
}

In [20]:
double k0;
double re,im;
std::complex<double> myexp(int ixp,int iyp, int m1, int n1, int m2, int n2, int il);

In [21]:
std::complex<double> myexp(int ixp,int iyp, int m1, int n1, int m2, int n2, int il) {
    k0 = 2*M_PI/(lmin+dl*il);
    re = pow((axmin+dax*ixp)/sxp,2)/4+pow((aymin+day*iyp)/syp,2)/4+pow(k0*Sx*dax*(m1-m2),2)
        *pow(k0*Sy*day*(n1-n2),2);
    im = k0*dx*dax*(m1-m2)*(axmin+dax*ixp)+k0*dy*day*(n1-n2)*(aymin+day*iyp);
    std::complex<double> ar(re, im);
    return exp(-ar);    
}

In [22]:
std::complex<double> s;

In [23]:
int64_t n;

In [24]:
int64_t m0;

In [25]:
double sz;

In [26]:
sz = 30e4;

In [20]:
std::srand(0)

In [21]:
std::rand()

1319830868

In [ ]:
m0 = 100000;
n = pow(10,9);
s = 0;
for (int64_t i=0;true;i++){
    il = std::rand() % nz;
    ixp = std::rand() % nx;
    m1 = std::rand() % nx;
    m2 = std::rand() % nx;
    iyp = std::rand() % ny;
    n1 = std::rand() % ny;
    n2 = std::rand() % ny;
    m1Mixp = m1-ixp;
    n1Miyp = n1-iyp;
    m2Mixp = m2-ixp;
    n2Miyp = n2-iyp;
    if (isin(m1Mixp, n1Miyp, m2Mixp, n2Miyp, il)) {
        s += ex3d(il,n1Miyp,m1Mixp)*ex3d(il,n1,m1)*ex3d(il,n2Miyp,m2Mixp)*ex3d(il,n2,m2)
            *myexp(ixp, iyp, m1, n1, m2, n2, il);
    }
    if (i == m0){
        std::complex<double> M = 1.0/(sqrt(M_PI)/sz*V/i/4.0/M_PI/sxp/syp*s);
        std::time_t t = std::time(nullptr);
        std::cout << std::put_time(std::localtime(&t), "%c %Z") << std::endl;
        std::cout << "n points = " << i << ", M = " << M << std::endl;
        m0 = 2*m0;
    }
}

Wed Jul 29 14:00:45 2020 CDT
n points = 100000, M = (5.99548e+08,-3.52271e+09)
Wed Jul 29 14:00:45 2020 CDT
n points = 200000, M = (1.20211e+09,-7.0464e+09)
Wed Jul 29 14:00:45 2020 CDT
n points = 400000, M = (4.30262e+09,-8.72833e+09)
Wed Jul 29 14:00:45 2020 CDT
n points = 800000, M = (8.28208e+09,-1.69238e+10)
Wed Jul 29 14:00:45 2020 CDT
n points = 1600000, M = (1.01988e+10,9.60541e+08)
Wed Jul 29 14:00:45 2020 CDT
n points = 3200000, M = (1.84575e+10,-1.47569e+10)
Wed Jul 29 14:00:46 2020 CDT
n points = 6400000, M = (3.24406e+10,-1.14181e+11)
Wed Jul 29 14:00:47 2020 CDT
n points = 12800000, M = (-5.56482e+09,-7.78202e+09)
Wed Jul 29 14:00:50 2020 CDT
n points = 25600000, M = (9.4276e+08,-1.73582e+10)
Wed Jul 29 14:00:55 2020 CDT
n points = 51200000, M = (-5.49328e+09,-8.16637e+09)
Wed Jul 29 14:01:05 2020 CDT
n points = 102400000, M = (1.53138e+10,-3.31138e+10)
Wed Jul 29 14:01:25 2020 CDT
n points = 204800000, M = (-8.06114e+09,1.56758e+10)
Wed Jul 29 14:02:05 2020 CDT
n points 

In [2]:
300*pow(700*1000,3)

1.0290000e+20